# Hybrid Recommendation System
**Goal**
- Combine User-CF, Item-CF, Content-CF, Model-CF, Trending models
- Adaptive weighting based on user scenario (new/cold/warm)
- Evaluate on warm-warm, warm-cold, cold-warm scenarios
- Save artifacts for API integration

### Task: Setup

In [ ]:
import os, sys, json, pickle, time
import numpy as np, polars as pl
from pathlib import Path
from scipy.sparse import load_npz
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..', 'utilities'))
sys.path.append(module_path)

from logger import Logger
from configurations import Configurations
from visualization_helpers import visualize_final_results
from evaluation_metrics import (
       recall_at_k,
       ndcg_at_k,
       map_at_k
)

m_log_file = Configurations.LOG_PATH
logger = Logger(process_name="hybrid", log_file=m_log_file)

PROCESSED_DIR = Path(Configurations.DATA_PROCESSED_PATH)
MODELS_DIR = Path(Configurations.MODELS_PATH)
CATEGORY = Configurations.CATEGORIES

logger.log_info("="*70)
logger.log_info("HYBRID RECOMMENDATION SYSTEM")
logger.log_info("="*70)
logger.log_info(f"Categories: {CATEGORY}")
logger.log_info(f"Sample size: {Configurations.DEV_SAMPLE_SIZE}")
logger.log_info("="*70 + "\n")

N_RECS = 10
COLD_USER_THRESHOLD = 5  # Users with < 5 ratings = cold

# %% [markdown]
## Helper Functions

# %%
def _candidate_files(category: str, split: str = "train"):
    dev_sample_size = Configurations.DEV_SAMPLE_SIZE
    safe_cat = category.replace('/', '-')
    
    if dev_sample_size != 'full':
        for size_name in Configurations.SAMPLE_SIZES.keys():
            if size_name == dev_sample_size:
                return PROCESSED_DIR / f"{safe_cat}.5core.{split}.{size_name}.parquet"
    else:
        return PROCESSED_DIR / f"{safe_cat}.5core.{split}.parquet"

def load_5core_data(category: str, split: str = "train") -> pl.DataFrame:
    p = _candidate_files(category, split)
    logger.log_info(f"[Load-{split.upper()}] {p}")
    df = pl.read_parquet(p, low_memory=False)
    logger.log_info(f"[Load-{split.upper()}] shape={df.shape} | "
                   f"users={df['user_id'].n_unique()} | "
                   f"items={df['parent_asin'].n_unique()}")
    return df

### Task: Load All Base Models

In [2]:
def load_all_models(category: str):
    """Load all 5 base models for hybrid"""
    logger.log_info(f"[Hybrid] Loading all models for {category}...")
    
    models = {}
    
    # Load Model-CF (SVD)
    model_dir = MODELS_DIR / 'model' / category
    if model_dir.exists():
        R = load_npz(model_dir / "R.npz")
        U = np.load(model_dir / "U.npy")
        V = np.load(model_dir / "V.npy")
        
        if V.shape[0] != U.shape[1]:
            logger.log_warning(f"V shape mismatch: {V.shape} (expected first dim = {U.shape[1]})")
            logger.log_info(f"Transposing V to correct shape...")
            V = V.T
            logger.log_info(f"V shape after transpose: {V.shape}")
        
        # Verify dimensions now match
        assert V.shape[0] == U.shape[1], f"U and V dimension mismatch: U{U.shape}, V{V.shape}"
        
        with open(model_dir / "user_rev.pkl", "rb") as f:
            user_rev = pickle.load(f)
        with open(model_dir / "item_rev.pkl", "rb") as f:
            item_rev = pickle.load(f)
        
        user_idx = json.loads((model_dir / "user_idx.json").read_text())
        item_idx = json.loads((model_dir / "item_idx.json").read_text())
        
        best_factors = Configurations.load_best_factors(category)
        
        models['model'] = {
            'R': R, 'U': U, 'V': V,
            'user_rev': user_rev, 'item_rev': item_rev,
            'user_idx': user_idx, 'item_idx': item_idx,
            'n_factors': best_factors
        }
        logger.log_info(f"  [4/5] Model-CF loaded (factors={best_factors}, U={U.shape}, V={V.shape})")
        
        # Load Item-CF
        item_dir = MODELS_DIR / 'item' / category
        if item_dir.exists():
            R = load_npz(item_dir / "R.npz")
            Rc = load_npz(item_dir / "Rc.npz") if (item_dir / "Rc.npz").exists() else None
            user_means = np.load(item_dir / "user_means.npy")
            item_similarity = load_npz(item_dir / "item_similarity.npz")
            
            with open(item_dir / "user_rev.pkl", "rb") as f:
                user_rev = pickle.load(f)
            with open(item_dir / "item_rev.pkl", "rb") as f:
                item_rev = pickle.load(f)
            
            user_idx = json.loads((item_dir / "user_idx.json").read_text())
            item_idx = json.loads((item_dir / "item_idx.json").read_text())
            
            best_k = Configurations.load_best_k_item(category)
            
            models['item'] = {
                'R': R, 'Rc': Rc, 'user_means': user_means,
                'item_similarity': item_similarity,
                'user_rev': user_rev, 'item_rev': item_rev,
                'user_idx': user_idx, 'item_idx': item_idx,
                'top_k_similar': best_k
            }
            logger.log_info(f"  [2/5] Item-CF loaded (K={best_k})")
        
        # Load Content-CF
        content_dir = MODELS_DIR / 'content' / category
        if content_dir.exists():
            R = load_npz(content_dir / "R.npz")
            item_similarity = load_npz(content_dir / "item_similarity.npz")
            
            with open(content_dir / "user_rev.pkl", "rb") as f:
                user_rev = pickle.load(f)
            with open(content_dir / "item_rev.pkl", "rb") as f:
                item_rev = pickle.load(f)
            
            user_idx = json.loads((content_dir / "user_idx.json").read_text())
            item_idx = json.loads((content_dir / "item_idx.json").read_text())
            
            best_k = Configurations.load_best_k_content(category)
            
            models['content'] = {
                'R': R, 'item_similarity': item_similarity,
                'user_rev': user_rev, 'item_rev': item_rev,
                'user_idx': user_idx, 'item_idx': item_idx,
                'top_k_similar': best_k
            }
            logger.log_info(f"  [3/5] Content-CF loaded (K={best_k})")
        
        # Load Model-CF (SVD)
        model_dir = MODELS_DIR / 'model' / category
        if model_dir.exists():
            R = load_npz(model_dir / "R.npz")
            U = np.load(model_dir / "U.npy")
            V = np.load(model_dir / "V.npy")
            
            with open(model_dir / "user_rev.pkl", "rb") as f:
                user_rev = pickle.load(f)
            with open(model_dir / "item_rev.pkl", "rb") as f:
                item_rev = pickle.load(f)
            
            user_idx = json.loads((model_dir / "user_idx.json").read_text())
            item_idx = json.loads((model_dir / "item_idx.json").read_text())
            
            best_factors = Configurations.load_best_factors(category)
            
            models['model'] = {
                'R': R, 'U': U, 'V': V,
                'user_rev': user_rev, 'item_rev': item_rev,
                'user_idx': user_idx, 'item_idx': item_idx,
                'n_factors': best_factors
            }
            logger.log_info(f"  [4/5] Model-CF loaded (factors={best_factors})")
        
        # Load Trending
        trending_dir = MODELS_DIR / 'trending' / category
        if trending_dir.exists():
            item_stats = pl.read_parquet(trending_dir / "item_stats.parquet")
            R = load_npz(trending_dir / "R.npz")
            
            with open(trending_dir / "user_rev.pkl", "rb") as f:
                user_rev = pickle.load(f)
            with open(trending_dir / "item_rev.pkl", "rb") as f:
                item_rev = pickle.load(f)
            
            user_idx = json.loads((trending_dir / "user_idx.json").read_text())
            item_idx = json.loads((trending_dir / "item_idx.json").read_text())
            
            models['trending'] = {
                'item_stats': item_stats,
                'R': R,
                'user_rev': user_rev, 'item_rev': item_rev,
                'user_idx': user_idx, 'item_idx': item_idx
            }
            logger.log_info(f"  [5/5] Trending loaded")
        
        logger.log_info(f"[Hybrid] Loaded {len(models)} models")
        
        return models


### Task: Scenario Detection

In [3]:
def detect_user_scenario(user_id: str, models: dict, threshold: int = 5):
    """
    Detect user scenario: new-user, cold-user, warm-user
    
    Returns: (scenario, n_ratings)
    """
    # Check in each model
    for model_name in ['user', 'item', 'content', 'model']:
        if model_name not in models:
            continue
        
        user_idx = models[model_name].get('user_idx', {})
        if user_id not in user_idx:
            continue
        
        R = models[model_name].get('R')
        if R is None:
            continue
        
        u = int(user_idx[user_id])
        n_ratings = R.getrow(u).nnz
        
        if n_ratings == 0:
            return 'new-user', 0
        elif n_ratings < threshold:
            return 'cold-user', n_ratings
        else:
            return 'warm-user', n_ratings
    
    return 'new-user', 0

### Task: Prediction Functions

In [4]:
def predict_user_cf(user_id: str, models: dict):
    """User-CF prediction"""
    if 'user' not in models:
        return None
    
    art = models['user']
    user_idx = art['user_idx']
    
    if user_id not in user_idx:
        return None
    
    u = int(user_idx[user_id])
    X = art['Rc'] if art.get('Rc') is not None else art['R']
    nn_model = art['nn_model']
    k = art.get('k_neighbors', 30)
    
    # Get similarities
    if isinstance(nn_model, dict) and 'similarity' in nn_model:
        sim_matrix = nn_model['similarity']
        user_sims = sim_matrix[u]
        top_k_idx = np.argsort(-user_sims)[:k]
        sims = user_sims[top_k_idx]
    else:
        distances, indices = nn_model.kneighbors(X.getrow(u), return_distance=True)
        d, idx = distances.ravel(), indices.ravel()
        mask = idx != u
        idx = idx[mask][:k]
        sims = np.clip(1.0 - d[mask][:k], 0.0, 1.0)
        top_k_idx = idx
    
    if len(top_k_idx) == 0:
        return None
    
    sims = np.clip(sims, 0, None)
    denom = np.sum(np.abs(sims)) + 1e-8
    scores = X[top_k_idx, :].T.dot(sims) / denom
    
    if art.get('Rc') is not None:
        scores = scores + art['user_means'][u]
    
    return scores


def predict_item_cf(user_id: str, models: dict):
    """Item-CF prediction"""
    if 'item' not in models:
        return None
    
    art = models['item']
    user_idx = art['user_idx']
    
    if user_id not in user_idx:
        return None
    
    u = int(user_idx[user_id])
    R = art['R']
    Rc = art.get('Rc')
    X = Rc if Rc is not None else R
    
    user_ratings = X.getrow(u).toarray().ravel()
    rated_items = np.nonzero(R.getrow(u).toarray().ravel())[0]
    
    if len(rated_items) == 0:
        return None
    
    item_sim = art['item_similarity']
    top_k = art.get('top_k_similar', 30)
    scores = np.zeros(R.shape[1], dtype=np.float32)
    
    sim_matrix = item_sim[:, rated_items].toarray()
    
    for i in range(R.shape[1]):
        sims = sim_matrix[i, :]
        positive_mask = sims > 0
        
        if not positive_mask.any():
            continue
        
        sims_pos = sims[positive_mask]
        ratings_pos = user_ratings[rated_items[positive_mask]]
        
        k_use = min(top_k, len(sims_pos))
        
        if k_use > 0:
            if k_use < len(sims_pos):
                top_idx = np.argpartition(-sims_pos, k_use-1)[:k_use]
            else:
                top_idx = np.arange(len(sims_pos))
            
            final_sims = sims_pos[top_idx]
            final_ratings = ratings_pos[top_idx]
            
            sim_sum = np.sum(final_sims)
            if sim_sum > 1e-8:
                scores[i] = np.dot(final_sims, final_ratings) / sim_sum
    
    if Rc is not None:
        scores = scores + art['user_means'][u]
    
    return scores


def predict_content_cf(user_id: str, models: dict):
    """Content-CF prediction"""
    if 'content' not in models:
        return None
    
    art = models['content']
    user_idx = art['user_idx']
    
    if user_id not in user_idx:
        return None
    
    u = int(user_idx[user_id])
    R = art['R']
    
    user_ratings = R.getrow(u).toarray().ravel()
    rated_items = np.nonzero(user_ratings)[0]
    
    if len(rated_items) == 0:
        return None
    
    item_sim = art['item_similarity']
    top_k = art.get('top_k_similar', 30)
    scores = np.zeros(R.shape[1], dtype=np.float32)
    
    sim_matrix = item_sim[:, rated_items].toarray()
    
    for i in range(R.shape[1]):
        sims = sim_matrix[i, :]
        positive_mask = sims > 0
        
        if not positive_mask.any():
            continue
        
        sims_pos = sims[positive_mask]
        ratings_pos = user_ratings[rated_items[positive_mask]]
        
        k_use = min(top_k, len(sims_pos))
        
        if k_use > 0:
            if k_use < len(sims_pos):
                top_idx = np.argpartition(-sims_pos, k_use-1)[:k_use]
            else:
                top_idx = np.arange(len(sims_pos))
            
            final_sims = sims_pos[top_idx]
            final_ratings = ratings_pos[top_idx]
            
            sim_sum = np.sum(final_sims)
            if sim_sum > 1e-8:
                scores[i] = np.dot(final_sims, final_ratings) / sim_sum
    
    return scores


def predict_model_cf(user_id: str, models: dict):
    if 'model' not in models:
        return None
    
    art = models['model']
    user_idx = art['user_idx']
    
    if user_id not in user_idx:
        return None
    
    u = int(user_idx[user_id])
    U = art['U']
    V = art['V']
    
    # Safety check: Ensure dimensions match
    # U[u] should be (n_factors,)
    # V should be (n_factors, n_items)
    if V.shape[0] == U.shape[1]:
        # V is correct shape (n_factors, n_items)
        scores = U[u] @ V
    else:
        # V is wrong shape, need transpose
        logger.log_warning(f"V shape mismatch in predict: U{U.shape}, V{V.shape}, transposing...")
        scores = U[u] @ V.T
    
    return scores


def get_trending_items(models: dict, n_items: int = 100):
    """Get trending items as array"""
    if 'trending' not in models:
        return None
    
    art = models['trending']
    item_stats = art['item_stats']
    
    # Get top trending
    top_items = item_stats['parent_asin'].to_list()[:n_items]
    
    # Convert to scores array matching matrix size
    item_idx = art['item_idx']
    scores = np.zeros(len(item_idx), dtype=np.float32)
    
    for rank, item in enumerate(top_items):
        if item in item_idx:
            idx = int(item_idx[item])
            # Reciprocal rank scoring
            scores[idx] = 1.0 / (rank + 1)
    
    return scores


### Task: Adaptive Weight Calculator

In [5]:
class AdaptiveWeights:    
    def __init__(self):
        self.configs = {
            'new-user': {
                'user': 0.0, 'item': 0.0, 'content': 0.0, 'model': 0.0, 'trending': 1.0
            },
            'cold-user': {
                'user': 0.1, 'item': 0.1, 'content': 0.3, 'model': 0.1, 'trending': 0.4
            },
            'warm-user': {
                'user': 0.25, 'item': 0.35, 'content': 0.20, 'model': 0.20, 'trending': 0.0
            }
        }
    
    def get_weights(self, scenario: str, available_models: list, n_ratings: int = 0):
        """Get adaptive weights"""
        base = self.configs.get(scenario, self.configs['warm-user'])
        
        # Filter to available
        weights = {m: w for m, w in base.items() if m in available_models}
        
        # Adjust by activity level
        if scenario == 'cold-user' and n_ratings > 0:
            # More ratings = more weight on CF
            if n_ratings >= 3:
                if 'user' in weights:
                    weights['user'] *= 1.5
                if 'item' in weights:
                    weights['item'] *= 1.5
                if 'trending' in weights:
                    weights['trending'] *= 0.7
        
        elif scenario == 'warm-user' and n_ratings > 20:
            # Very active user = more weight on CF
            if 'user' in weights:
                weights['user'] *= 1.2
            if 'item' in weights:
                weights['item'] *= 1.2
        
        # Normalize
        total = sum(weights.values())
        if total > 0:
            weights = {k: v / total for k, v in weights.items()}
        
        return weights

### Task: Hybrid Prediction

In [6]:
def predict_hybrid(user_id: str, models: dict, weight_calc: AdaptiveWeights):
    # Detect scenario
    scenario, n_ratings = detect_user_scenario(user_id, models, COLD_USER_THRESHOLD)
    
    logger.log_info(f"[Hybrid] User {user_id[:12]}... | Scenario: {scenario} | Ratings: {n_ratings}")
    
    # Get predictions from each model
    predictions = {}
    
    user_scores = predict_user_cf(user_id, models)
    if user_scores is not None:
        predictions['user'] = user_scores
    
    item_scores = predict_item_cf(user_id, models)
    if item_scores is not None:
        predictions['item'] = item_scores
    
    content_scores = predict_content_cf(user_id, models)
    if content_scores is not None:
        predictions['content'] = content_scores
    
    model_scores = predict_model_cf(user_id, models)
    if model_scores is not None:
        predictions['model'] = model_scores
    
    trending_scores = get_trending_items(models, n_items=100)
    if trending_scores is not None:
        predictions['trending'] = trending_scores
    
    if not predictions:
        logger.log_warning(f"[Hybrid] No predictions available for {user_id}")
        return None, "no_models", {}
    
    # Get weights
    available = list(predictions.keys())
    weights = weight_calc.get_weights(scenario, available, n_ratings)
    
    logger.log_info(f"[Hybrid] Models: {available}")
    logger.log_info(f"[Hybrid] Weights: {format_weights(weights)}")
    
    # Combine predictions
    ref_model = available[0]
    n_items = len(predictions[ref_model])
    combined = np.zeros(n_items, dtype=np.float32)
    
    for model_name, scores in predictions.items():
        weight = weights.get(model_name, 0.0)
        if weight > 0:
            combined += weight * scores
    
    # Build strategy string
    strategy_parts = [f"{m}({w*100:.0f}%)" for m, w in sorted(weights.items(), key=lambda x: -x[1]) if w > 0.05]
    strategy = f"hybrid-{scenario}-" + "+".join(strategy_parts)
    
    metadata = {
        'scenario': scenario,
        'n_ratings': n_ratings,
        'weights': weights,
        'models': available
    }
    
    return combined, strategy, metadata


def format_weights(weights: dict) -> str:
    """Format weights for logging"""
    return "{" + ", ".join(f"{k}:{v:.2f}" for k, v in weights.items()) + "}"

### Task: Recommendation

In [7]:
def recommend_hybrid(user_id: str, n_recs: int, models: dict, weight_calc: AdaptiveWeights):    
    # Predict
    scores, strategy, metadata = predict_hybrid(user_id, models, weight_calc)
    
    if scores is None:
        logger.log_warning(f"[Recommend] No scores for {user_id}")
        return pl.DataFrame({"parent_asin": [], "score": []}), strategy
    
    # Get reference R and item_rev
    ref_model = metadata['models'][0]
    R = models[ref_model]['R']
    item_rev = models[ref_model]['item_rev']
    user_idx = models[ref_model]['user_idx']
    
    # Exclude rated items
    rated = set()
    if user_id in user_idx:
        u = int(user_idx[user_id])
        rated = set(R.getrow(u).indices.tolist())
    
    cand_mask = np.ones(len(scores), dtype=bool)
    if rated:
        cand_mask[list(rated)] = False
    
    cand_scores = scores[cand_mask]
    
    if cand_scores.size == 0:
        return pl.DataFrame({"parent_asin": [], "score": []}), strategy
    
    # Select top-N
    n_top = min(n_recs, cand_scores.size)
    cand_indices = np.nonzero(cand_mask)[0]
    top_pos = np.argpartition(-cand_scores, n_top - 1)[:n_top]
    top_pos = top_pos[np.argsort(-cand_scores[top_pos])]
    
    rec_asins = [item_rev[cand_indices[i]] for i in top_pos]
    rec_scores = [float(cand_scores[i]) for i in top_pos]
    
    return pl.DataFrame({"parent_asin": rec_asins, "score": rec_scores}), strategy


### Task: Evaluation Pipeline

In [8]:
def evaluate_hybrid(category: str, models: dict, weight_calc: AdaptiveWeights,
                   k_values: list = [10, 20, 50], split: str = "test", 
                   sample_users: int = 1000):
    
    logger.log_info(f"[Eval-Hybrid] {category} on {split.upper()}")
    
    df_eval = load_5core_data(category, split=split)
    
    # Get train users from any model
    train_users = None
    for model_name in ['user', 'item', 'content', 'model']:
        if model_name in models:
            train_users = list(models[model_name]['user_idx'].keys())
            break
    
    if train_users is None:
        logger.log_warning("[Eval-Hybrid] No models available")
        return None
    
    # Filter to train users
    df_eval = df_eval.filter(pl.col('user_id').is_in(train_users))
    
    if len(df_eval) == 0:
        logger.log_warning("[Eval-Hybrid] No data after filtering")
        return None
    
    logger.log_info(f"[Eval-Hybrid] {len(df_eval):,} ratings, {df_eval['user_id'].n_unique():,} users")
    
    # Sample users
    eval_users = df_eval['user_id'].unique().to_list()
    if len(eval_users) > sample_users:
        np.random.seed(42)
        eval_users = np.random.choice(eval_users, sample_users, replace=False).tolist()
    
    logger.log_info(f"[Eval-Hybrid] Evaluating {len(eval_users)} users...")
    
    # Initialize metrics
    metrics_acc = {
        **{f'recall@{k}': [] for k in k_values},
        **{f'ndcg@{k}': [] for k in k_values},
        **{f'map@{k}': [] for k in k_values}
    }
    
    scenario_counts = {'new-user': 0, 'cold-user': 0, 'warm-user': 0}
    evaluated = 0
    
    for user_id in eval_users:
        # Get relevant items
        user_eval = df_eval.filter(pl.col('user_id') == user_id)
        relevant = set(user_eval.filter(pl.col('rating') >= 4)['parent_asin'].to_list())
        
        if len(relevant) == 0:
            continue
        
        # Get recommendations
        recs_df, strategy = recommend_hybrid(user_id, n_recs=50, models=models, weight_calc=weight_calc)
        
        if len(recs_df) == 0:
            continue
        
        recommended = recs_df['parent_asin'].to_list()
        
        # Track scenario
        scenario = strategy.split('-')[1] if len(strategy.split('-')) > 1 else 'unknown'
        scenario_counts[scenario] = scenario_counts.get(scenario, 0) + 1
        
        evaluated += 1
        
        # Calculate metrics
        for k in k_values:
            metrics_acc[f'recall@{k}'].append(recall_at_k(recommended, relevant, k))
            metrics_acc[f'ndcg@{k}'].append(ndcg_at_k(recommended, relevant, k))
            metrics_acc[f'map@{k}'].append(map_at_k(recommended, relevant, k))
    
    logger.log_info(f"[Eval-Hybrid] Evaluated: {evaluated} users")
    logger.log_info(f"[Eval-Hybrid] Scenarios: {scenario_counts}")
    
    # Aggregate
    results = {
        'category': category,
        'split': split,
        'n_users': evaluated,
        'rmse': np.nan,  # Hybrid uses ranking, not rating prediction
        'accuracy': np.nan
    }
    
    for k in k_values:
        for metric in ['recall', 'ndcg', 'map']:
            key = f'{metric}@{k}'
            results[key] = np.mean(metrics_acc[key]) if metrics_acc[key] else 0.0
    
    logger.log_info(f"[Eval-Hybrid] NDCG@10={results['ndcg@10']:.4f}, "
                   f"Recall@10={results['recall@10']:.4f}")
    
    return results

### Task: Save/Load Artifacts

In [9]:
def save_hybrid_config(out_dir: Path, weight_calc: AdaptiveWeights, metadata: dict):
    out_dir.mkdir(parents=True, exist_ok=True)
    
    config = {
        'weight_configs': weight_calc.configs,
        'cold_threshold': COLD_USER_THRESHOLD,
        'metadata': metadata
    }
    
    with open(out_dir / 'config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    logger.log_info(f"[Saved-Hybrid] {out_dir}")


def load_hybrid_config(model_dir: Path):
    """Load hybrid configuration"""
    with open(model_dir / 'config.json', 'r') as f:
        return json.load(f)


### Task: Main Pipeline

In [10]:
def run_hybrid_pipeline(category: str):
    """Main pipeline for hybrid system"""
    
    logger.log_info(f"\n{'='*70}\nCATEGORY: {category}\n{'='*70}\n")
    
    model_dir = MODELS_DIR / 'hybrid' / category
    
    # Load all base models
    logger.log_info("[STEP 1] Loading base models...")
    models = load_all_models(category)
    
    if len(models) < 2:
        logger.log_warning(f"[Skip] Not enough models for {category} (need >= 2)")
        return None
    
    # Initialize weight calculator
    logger.log_info("\n[STEP 2] Initializing adaptive weights...")
    weight_calc = AdaptiveWeights()
    
    # Evaluate on validation
    logger.log_info("\n[STEP 3] Evaluating on VALIDATION...")
    val_results = evaluate_hybrid(
        category, models, weight_calc,
        k_values=[10, 20, 50],
        split="valid",
        sample_users=Configurations.get_eval_samples_tuning()
    )
    
    # Evaluate on test
    logger.log_info("\n[STEP 4] Evaluating on TEST...")
    test_results = evaluate_hybrid(
        category, models, weight_calc,
        k_values=[10, 20, 50],
        split="test",
        sample_users=Configurations.get_eval_samples_final()
    )
    
    # Save config
    logger.log_info("\n[STEP 5] Saving configuration...")
    save_hybrid_config(model_dir, weight_calc, {
        'models_used': list(models.keys()),
        'category': category
    })
    
    logger.log_info(f"\n[Complete] Hybrid system ready for {category}")
    logger.log_info("="*70 + "\n")
    
    return {
        'val_results': val_results,
        'test_results': test_results,
        'models_used': list(models.keys())
    }

### Task: Execute Pipeline

In [ ]:
logger.log_info("\n" + "="*70)
logger.log_info("HYBRID SYSTEM - ALL CATEGORIES")
logger.log_info("="*70 + "\n")

hybrid_results = {}

for cat in CATEGORY:
    try:
        result = run_hybrid_pipeline(cat)
        if result:
            hybrid_results[cat] = result
    except Exception as e:
        logger.log_exception(f"[Error] {cat}: {e}")

### Task: Save Final Results

In [ ]:
logger.log_info("\n" + "="*70)
logger.log_info("SAVING RESULTS")
logger.log_info("="*70 + "\n")

test_results_list = [hybrid_results[cat]['test_results'] 
                     for cat in CATEGORY 
                     if cat in hybrid_results and hybrid_results[cat]['test_results']]

if test_results_list:
    df_final = pl.DataFrame(test_results_list)
    
    logger.log_info("Final Test Results:")
    display(df_final)
    
    out_csv = MODELS_DIR / 'hybrid' / 'final_test_results.csv'
    df_final.write_csv(out_csv)
    logger.log_info(f"\nSaved: {out_csv}")
    
    visualize_final_results(
        test_results_list,
        save_dir=MODELS_DIR / 'hybrid',
        algo_name='Hybrid',
        k_values=[10, 20, 50]
    )
    logger.log_info("Saved: evaluation_results.png\n")

logger.log_info("="*70)
logger.log_info("HYBRID SYSTEM COMPLETE")
logger.log_info("="*70 + "\n")

### Task: Compare all models

In [ ]:
def compare_all_models():
    """Compare all algorithms on test set"""
    
    logger.log_info("\n" + "="*70)
    logger.log_info("COMPARISON: ALL ALGORITHMS")
    logger.log_info("="*70 + "\n")
    
    results = []
    
    for algo in ['user', 'item', 'content', 'model', 'trending', 'hybrid']:
        file = MODELS_DIR / algo / 'final_test_results.csv'
        
        if file.exists():
            df = pl.read_csv(file)
            df = df.with_columns(pl.lit(algo).alias('algorithm'))
            results.append(df)
            logger.log_info(f"[Load] {algo}: {len(df)} categories")
    
    if not results:
        logger.log_warning("No results found")
        return
    
    df_all = pl.concat(results)
    
    # Display comparison
    logger.log_info("\nAll Results:")
    display(df_all.select(['algorithm', 'category', 'ndcg@10', 'recall@10', 'map@10']))
    
    # Save
    out_csv = MODELS_DIR / 'comparison_all_models.csv'
    df_all.write_csv(out_csv)
    logger.log_info(f"\nSaved: {out_csv}")
    
    # Visualize comparison
    plot_model_comparison(df_all)
    
    return df_all

def plot_model_comparison(df_all: pl.DataFrame):
    """Plot comparison of all models"""
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    algorithms = df_all['algorithm'].unique().to_list()
    metrics = ['ndcg@10', 'recall@10', 'map@10']
    titles = ['NDCG@10', 'Recall@10', 'MAP@10']
    
    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        scores = []
        for algo in algorithms:
            algo_data = df_all.filter(pl.col('algorithm') == algo)
            if len(algo_data) > 0:
                scores.append(algo_data[metric][0])
            else:
                scores.append(0)
        
        axes[idx].bar(algorithms, scores, alpha=0.8)
        axes[idx].set_ylabel('Score')
        axes[idx].set_title(title, fontweight='bold')
        axes[idx].set_xticks(axes[idx].get_xticks())
        axes[idx].set_xticklabels(algorithms, rotation=45)
        axes[idx].grid(axis='y', alpha=0.3)
        
        # Add values on bars
        for i, v in enumerate(scores):
            axes[idx].text(i, v, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    plt.suptitle('Model Comparison - All Algorithms', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    out_path = MODELS_DIR / 'model_comparison.png'
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    logger.log_info(f"Saved comparison plot: {out_path}")
    plt.show()

compare_all_models()


### Task: Unit test

In [ ]:
def test_hybrid_recommendation():
    """Unit test: Verify hybrid works"""
    
    logger.log_info("\n" + "="*70)
    logger.log_info("[UNIT TEST] Hybrid Recommendation")
    logger.log_info("="*70 + "\n")
    
    cat = CATEGORY[0]
    
    # Load models
    models = load_all_models(cat)
    weight_calc = AdaptiveWeights()
    
    # Get test users
    user_rev = models[list(models.keys())[0]]['user_rev']
    
    # Test different user types
    test_cases = [
        ('First user', user_rev[0]),
        ('Middle user', user_rev[len(user_rev)//2]),
        ('Last user', user_rev[-1])
    ]
    
    for case_name, user_id in test_cases:
        logger.log_info(f"\n[Test] {case_name}: {user_id[:12]}...")
        logger.log_info("-"*70)
        
        try:
            recs_df, strategy = recommend_hybrid(user_id, N_RECS, models, weight_calc)
            
            logger.log_info(f"  Strategy: {strategy}")
            logger.log_info(f"  Recommendations: {len(recs_df)}")
            
            if len(recs_df) > 0:
                logger.log_info(f"  Score range: [{recs_df['score'].min():.2f}, {recs_df['score'].max():.2f}]")
                display(recs_df.head(5))
        
        except Exception as e:
            logger.log_exception(f"  Error: {e}")
    
    logger.log_info("\n" + "="*70)
    logger.log_info("UNIT TEST COMPLETE")
    logger.log_info("="*70 + "\n")

test_hybrid_recommendation()